In [57]:
import json
import requests
from bs4 import BeautifulSoup as BS

In [53]:
# list of urls
d = {}
urls = ['https://en.wikipedia.org/wiki/Alibaba_Group', 'https://en.wikipedia.org/wiki/Amazon_(company)', 'https://en.wikipedia.org/wiki/EBay']

# layout of wiki page is standardized
# text content is all in p tags
for u in urls:
    
    # dictionary to to hold text information by url
    # in case we try and crawl all on a central notebook
    d[u] = []
    wiki_pg = requests.get(u)
    soup = BS(wiki_pg.content, 'lxml')
    toc = str(soup.find(id='toc'))
    
    soup_toc = BS(toc, 'lxml')
    
    toc_num = soup.find_all('span', {"class" : 'tocnumber'})
    toc_data = soup.find_all('span', {"class" : 'toctext'})
    
    d[u] = (toc_num, toc_data)

In [58]:
for u in d.keys():
    
    toc_dict = {}
    toc_num, toc_data = d[u]
    
    # generate list of keys
    keys = {}
    for i in range(len(toc_num)):
        
        if '.' not in toc_num[i].text:
            keys[toc_num[i].text] = toc_data[i].text
    
    # actually add to dictionary now
    for i in range(len(toc_num)):
        
        if '.' not in toc_num[i].text:
            toc_dict[toc_data[i].text] = []
            continue
            
        num = toc_num[i].text.split('.')[0]
        key = keys[num]
        
        if key in toc_dict:
            toc_dict[key].append(toc_data[i].text)
        else:
            toc_dict[key] = [toc_data[i].text]
        
    print (u)
    print (json.dumps(toc_dict))
    print ('######')

https://en.wikipedia.org/wiki/EBay
{"Controversy and criticism": ["2014 security breach"], "See also": [], "History": ["1990s", "2000s", "2010s"], "External links": [], "References": [], "Corporate affairs": ["Board of directors", "Logo", "Profit and transactions", "Environmental record", "Acquisitions", "PayPal", "Craigslist", "Skype", "StubHub", "Corrigon", "Qoo10.jp"], "Use for data analysis": ["Third party", "Visual search", "GCHQ", "Items", "Unusual items", "Prohibited or restricted items", "Bidding", "Auction-style listings", "Seller ratings", "Charity auctions", "Shipping"]}
######
https://en.wikipedia.org/wiki/Amazon_(company)
{"External links": [], "See also": [], "History": [], "Products and services": [], "Further reading": [], "Merchant partnerships": [], "Multi-level sales strategy": [], "Subsidiaries": ["A9.com", "Amazon Maritime", "Audible.com", "Beijing Century Joyo Courier Services", "Brilliance Audio", "ComiXology", "CreateSpace", "Eero", "Goodreads", "Health Navigato